In [1]:
import setigen as stg
import bifrost as bf
from bifrost.fdmt import Fdmt
import numpy as np
import blimpy as bl
import cupy as cp
import matplotlib.pyplot as plt
from astropy import units as u

import hyperseti
from hyperseti import dedoppler
from hyperseti import plotting 

import time as tim
from timeit import default_timer as timer

%matplotlib inline

fdmt= Fdmt()


/home/cho134/.local/lib/python3.9/site-packages/cupy/_environment.py:434: UserWarning: 
--------------------------------------------------------------------------------

  CuPy may not function correctly because multiple CuPy packages are installed
  in your environment:

    cupy, cupy-cuda110

  Follow these steps to resolve this issue:

    1. For all packages listed above, run the following command to remove all
       existing CuPy installations:

         $ pip uninstall <package_name>

      If you previously installed CuPy via conda, also run the following:

         $ conda uninstall cupy

    2. Install the appropriate CuPy package.
       Refer to the Installation Guide for detailed instructions.

         https://docs.cupy.dev/en/stable/install.html

--------------------------------------------------------------------------------

  warnings.warn(f'''
bshuf filter already loaded, skip it.


RuntimeError: b'BF_STATUS_DEVICE_ERROR'

In [ ]:
def framegen(fchans,tchans,df,dt,fch1):
    frame = stg.Frame(fchans=fchans*u.pixel,
                      tchans=tchans*u.pixel,
                      df=df*u.Hz,
                      dt=dt*u.s,
                      fch1=fch1*u.MHz)
    noise = frame.add_noise(x_mean=10, noise_type='chi2')
    signal = frame.add_signal(stg.constant_path(f_start=frame.get_frequency(index=frame.fchans/2),
                                                drift_rate=-frame.df/frame.dt),
                              stg.constant_t_profile(level=frame.get_intensity(snr=30)),
                              stg.box_f_profile(width=1*u.Hz),
                              stg.constant_bp_profile(level=1))
    return frame

In [ ]:
def ddframecenteredtwice(stgframe, exponent):
    fr=stgframe
    drift_rate=fr.df/fr.dt
    frequency = fr.dt*fr.tchans*drift_rate
    max_delay=int(frequency*2)

    d_cpu = np.expand_dims(fr.data, axis=0)
    ## Initialize FDMT
    n_disp = max_delay
    n_time = d_cpu.shape[2]
    n_chan = d_cpu.shape[1]
    fdmt.init(n_chan, n_disp, fr.fch1, fr.df, space="cuda", exponent=exponent)
    # Input shape is (1, n_freq, n_time)
    d_in = bf.ndarray(d_cpu, dtype='f32', space='cuda')
    d_out = bf.ndarray(np.zeros(shape=(1, n_disp, n_time)), dtype='f32', space='cuda')
    d_out2 = bf.ndarray(np.zeros(shape=(1, n_disp, n_time)), dtype='f32', space='cuda')
    # Execute FDMT 
    start = timer()
    for ii in range(128):
        fdmt.execute(d_in, d_out, negative_delays=True)
        fdmt.execute(d_in, d_out2, negative_delays=False)
    end = timer()
    time=end-start
    timesFDMT.append(time)
    
    d_out = d_out.copy(space='system')
    d_out2 = d_out2.copy(space='system')
    
    img = cp.asarray(d_out2.squeeze())
    peaks, xvals, yvals = hyperseti.peak.prominent_peaks(img, num_peaks=1, min_xdistance=100, min_ydistance=100, threshold=100)
    print(peaks)    

    
def hypersetidd(frame):
    fr=frame
    drift_rate=fr.df/fr.dt
    frequency = fr.dt*fr.tchans*drift_rate
    max_delay=int(frequency*2)
    
    test_data = np.expand_dims(fr.data.astype('float32'), axis=1)
    test_data=cp.array(test_data)
    metadata = {'frequency_start': frame.get_frequency(index=fr.fchans/2)*u.MHz, 'time_step': fr.dt*u.s, 'frequency_step': fr.df*u.Hz}

    # Run dedoppler
    start = timer()
    dedopp, metadata= dedoppler(test_data, metadata, boxcar_size=1, max_dd=(max_delay/fr.tchans)*(fr.df/fr.dt), return_space='gpu')
    end = timer()
    timesHyperseti.append(end - start)